In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error


In [55]:
train = pd.read_csv("datasets/train.csv.zip", compression="zip", index_col="id")
test = pd.read_csv("datasets/test.csv", index_col="id")


In [56]:
oil = pd.read_csv("datasets/oil.csv", index_col="date", parse_dates=["date"])
oil["price"] = oil["dcoilwtico"]
oil.drop("dcoilwtico", inplace=True, axis="columns")


In [57]:
train.head()


,date,store_nbr,family,sales,onpromotion
id,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,2013-01-01,1,BABY CARE,0.0,0
2,2013-01-01,1,BEAUTY,0.0,0
3,2013-01-01,1,BEVERAGES,0.0,0
4,2013-01-01,1,BOOKS,0.0,0


In [58]:
test.head()


,date,store_nbr,family,onpromotion
id,,,,
3000888,2017-08-16,1,AUTOMOTIVE,0
3000889,2017-08-16,1,BABY CARE,0
3000890,2017-08-16,1,BEAUTY,2
3000891,2017-08-16,1,BEVERAGES,20
3000892,2017-08-16,1,BOOKS,0


In [59]:
train["Time"] = np.arange(len(train.index))
encoder = LabelEncoder()
train["category"] = encoder.fit_transform(train.family)

In [60]:
X = train.drop(["sales", "date", "family"], axis="columns")
y = train.sales

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [68]:
model = RandomForestRegressor(n_estimators=20, max_depth=10)
model.fit(X_train, y_train)


RandomForestRegressor(max_depth=10, n_estimators=20)

In [69]:
y_pred = model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred)
print("Error is ", rmse)

Error is  202496.51205666756


In [70]:
test["Time"] = np.arange(len(train.index), len(train.index) + len(test.index))
test["category"] = encoder.transform(test.family)
X_test = test.drop(["date", "family"], axis="columns")
X_test.head()

,store_nbr,onpromotion,Time,category
id,,,,
3000888,1,0,3000888,0
3000889,1,0,3000889,1
3000890,1,2,3000890,2
3000891,1,20,3000891,3
3000892,1,0,3000892,4


In [71]:
y_test = model.predict(x_test)
submission = pd.DataFrame({"sales": y_test}, index=test.index)
submission.to_csv("datasets/submission.csv")
